## Model Experimentation, Training, and Evaluation for Flood Prediction

### Introduction
This notebook focuses on training and evaluating machine learning models for flood prediction, utilizing the engineered features from `02_feature_engineering_exploration.ipynb`. We will cover data splitting (time-series), feature scaling, model selection, hyperparameter tuning, and comprehensive evaluation metrics.

### 1. Setup and Load Processed Data

First, load the processed DataFrame that contains all your engineered features.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier # Add other models as you experiment
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    precision_recall_curve,
    auc,
    RocCurveDisplay # For plotting ROC curve
)
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from imblearn.over_sampling import SMOTE # For handling imbalance, if chosen
from imblearn.pipeline import Pipeline as ImbPipeline # Use imblearn's pipeline for SMOTE if using SMOTE

import joblib # For saving/loading models and scalers

# Configure matplotlib for better aesthetics
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('viridis')

# Define path to processed data
PROCESSED_DATA_PATH = '../data/processed/'
PROCESSED_DATA_FILE = 'merged_processed_data.csv'

try:
    df_processed = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, PROCESSED_DATA_FILE), index_col='timestamp', parse_dates=True)
    df_processed = df_processed.sort_index()
    print(f"Processed data loaded. Shape: {df_processed.shape}")
    print("Processed Data Head:")
    print(df_processed.head())
except FileNotFoundError:
    print(f"Error: {PROCESSED_DATA_FILE} not found. Please ensure '02_feature_engineering_exploration.ipynb' was run successfully.")
    print("Exiting notebook. Please run the previous notebook first.")
    exit() # Exit if the processed data isn't found

# Display flood event distribution
print("\nFlood event value counts in processed data:")
print(df_processed['flood_event'].value_counts())
print(f"Dataset range: {df_processed.index.min().strftime('%Y-%m-%d')} to {df_processed.index.max().strftime('%Y-%m-%d')}")

### 2. Data Splitting (Time-Series)

Crucially, we'll split our data chronologically. Given your data range (2023-04-30 to 2025-04-30), we'll use the first 1.5 years for training and the last 0.5 year for testing.

In [ ]:
print("\n--- 2. Data Splitting (Time-Series) ---")

# Define features (X) and target (y)
X = df_processed.drop('flood_event', axis=1)
y = df_processed['flood_event']

# Define the split date
# Train: 2023-04-30 to 2024-10-29 (approx. 1.5 years)
# Test:  2024-10-30 to 2025-04-30 (approx. 0.5 years)
split_date = pd.to_datetime('2024-10-30')

X_train = X[X.index < split_date]
y_train = y[y.index < split_date]

X_test = X[X.index >= split_date]
y_test = y[y.index >= split_date]

print(f"Training data range: {X_train.index.min().strftime('%Y-%m-%d')} to {X_train.index.max().strftime('%Y-%m-%d')}")
print(f"Testing data range: {X_test.index.min().strftime('%Y-%m-%d')} to {X_test.index.max().strftime('%Y-%m-%d')}")
print(f"Training data shape: {X_train.shape}, Target shape: {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, Target shape: {y_test.shape}")
print(f"Flood events in training: {y_train.sum()} ({y_train.sum()/len(y_train)*100:.2f}%)")
print(f"Flood events in testing: {y_test.sum()} ({y_test.sum()/len(y_test)*100:.2f}%)")

### 3. Feature Scaling

We'll use StandardScaler to normalize our numerical features. Remember to fit on training data and transform both training and test data.

In [ ]:
print("\n--- 3. Feature Scaling ---")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for easier inspection and consistency (optional but good practice)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("Features scaled.")
print("Example of scaled data (first 5 rows):")
print(X_train_scaled.head())

### 4. Model Selection and Hyperparameter Tuning (Random Forest)

We'll focus on a Random Forest Classifier as a strong baseline, using GridSearchCV with TimeSeriesSplit for robust tuning. Remember that for flood prediction, Recall is often the most critical metric (minimizing false negatives).

In [ ]:
print("\n--- 4. Model Selection and Hyperparameter Tuning (Random Forest) ---")

# Define the model to tune
rf_model = RandomForestClassifier(random_state=42)

# Define the parameter grid to search
param_grid_rf = {
    'n_estimators': [50, 100, 150, 200], # Number of trees
    'max_depth': [None, 10, 20],      # Max depth of trees
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],    # Minimum samples required at a leaf node
    'class_weight': ['balanced', None] # 'balanced' is crucial for imbalance, None for comparison
}

# TimeSeriesSplit for robust cross-validation
# For a 1.5 year training set, 3 splits is a reasonable starting point.
# Each fold will train on ~0.5 years and validate on the subsequent ~0.5 years.
tscv = TimeSeriesSplit(n_splits=3)

# GridSearchCV setup
grid_search_rf = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid_rf,
    cv=tscv,              # Use time-series cross-validation
    scoring='recall',     # Optimize for recall (minimizing missed floods)
    n_jobs=-1,            # Use all available CPU cores
    verbose=2,            # Show detailed progress
    error_score='raise'   # Raise errors explicitly
)

print("Starting GridSearchCV for RandomForestClassifier...")
grid_search_rf.fit(X_train_scaled, y_train)

best_rf_model = grid_search_rf.best_estimator_
print(f"\nBest Hyperparameters for RandomForest: {grid_search_rf.best_params_}")
print(f"Best cross-validation Recall Score: {grid_search_rf.best_score_:.4f}")

# Save the best model and scaler
joblib.dump(best_rf_model, '../models/trained_models/RandomForestClassifier_v1.pkl')
joblib.dump(scaler, '../models/trained_models/StandardScaler_v1.pkl')
print("\nBest RandomForest model and scaler saved to 'models/trained_models/'.")

### 5. Model Evaluation on the Unseen Test Set

Now, we evaluate the best_rf_model on our completely unseen test set. This gives us the most realistic estimate of its real-world performance.

In [ ]:
print("\n--- 5. Model Evaluation on Unseen Test Set ---")

# Predictions on the test set
y_pred_rf = best_rf_model.predict(X_test_scaled)
y_pred_proba_rf = best_rf_model.predict_proba(X_test_scaled)[:, 1] # Probabilities for the positive class (flood)

print("\n--- Classification Report (Random Forest) ---")
print(classification_report(y_test, y_pred_rf))

print("\n--- Confusion Matrix (Random Forest) ---")
cm_rf = confusion_matrix(y_test, y_pred_rf)
print(cm_rf)

plt.figure(figsize=(6, 5))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicted No Flood', 'Predicted Flood'],
            yticklabels=['Actual No Flood', 'Actual Flood'])
plt.title('Confusion Matrix (Random Forest)')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.show()
plt.savefig('../reports/figures/rf_confusion_matrix.png')


print(f"\nROC AUC Score (Random Forest): {roc_auc_score(y_test, y_pred_proba_rf):.4f}")

# Precision-Recall Curve (Crucial for Imbalanced Data)
precision_rf, recall_rf, _ = precision_recall_curve(y_test, y_pred_proba_rf)
pr_auc_rf = auc(recall_rf, precision_rf)
print(f"Precision-Recall AUC (Random Forest): {pr_auc_rf:.4f}")

plt.figure(figsize=(8, 6))
plt.plot(recall_rf, precision_rf, label=f'Precision-Recall curve (AUC = {pr_auc_rf:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (Random Forest)')
plt.legend(loc='lower left')
plt.grid(True)
plt.show()
plt.savefig('../reports/figures/rf_pr_curve.png')

# Plot ROC Curve
RocCurveDisplay.from_estimator(best_rf_model, X_test_scaled, y_test, name='Random Forest')
plt.title('ROC Curve (Random Forest)')
plt.grid(True)
plt.show()
plt.savefig('../reports/figures/rf_roc_curve.png')

# Feature Importance
if hasattr(best_rf_model, 'feature_importances_'):
    importances_rf = best_rf_model.feature_importances_
    feature_names = X_train.columns
    sorted_indices_rf = np.argsort(importances_rf)[::-1]

    plt.figure(figsize=(10, 6))
    sns.barplot(x=importances_rf[sorted_indices_rf], y=feature_names[sorted_indices_rf])
    plt.title("Feature Importances (Random Forest)")
    plt.xlabel("Importance")
    plt.ylabel("Feature")
    plt.tight_layout()
    plt.show()
    plt.savefig('../reports/figures/rf_feature_importance.png')

    print("\nTop 5 Most Important Features (Random Forest):")
    for i in sorted_indices_rf[:5]:
        print(f"- {feature_names[i]}: {importances_rf[i]:.4f}")

### 6. Summary and Next Steps

This notebook provided a comprehensive workflow for training and evaluating a flood prediction model. We performed a time-series split, scaled features, tuned a Random Forest Classifier, and thoroughly evaluated its performance using relevant metrics like Recall and PR AUC.

### Key Learnings from this Notebook:

- How well the model performed on the unseen test set (check the confusion matrix and classification report).
- Which features were most important according to the model.
- The trade-offs between precision and recall shown by the PR curve.
### Next Steps:

- If performance is not satisfactory, consider experimenting with other models (e.g., Gradient Boosting like XGBoost/LightGBM, Logistic Regression as a baseline).
- Refine feature engineering further based on model performance and feature importances.
- Discuss the results with domain experts to validate findings and understand actionable insights.
- Move the finalized model training and evaluation logic into reusable scripts in the src/ directory for production.